In [1]:
import math

from mindspore import nn
from mindspore import ops
from mindspore.common.initializer import initializer

from mindnlp import load_dataset, process, Vocab
from mindnlp._legacy.abc import Seq2vecModel
from mindnlp.engine import Trainer
from mindnlp.metrics import Accuracy
from mindnlp.modules import Glove, RNNEncoder, StaticLSTM
from mindnlp.transforms import BasicTokenizer

In [2]:
# load datasets
imdb_train, imdb_test = load_dataset('imdb', shuffle=True)
print(imdb_train.get_col_names())

In [3]:
tokenizer = BasicTokenizer(True)
vocab = Vocab.from_pretrained(name="glove.6B.100d")

imdb_train = process('imdb', imdb_train, tokenizer=tokenizer, vocab=vocab, \
                     bucket_boundaries=[400, 500], max_len=600, drop_remainder=True)
imdb_test = process('imdb', imdb_test, tokenizer=tokenizer, vocab=vocab, \
                     bucket_boundaries=[400, 500], max_len=600, drop_remainder=False)

In [4]:
class SentimentClassification(Seq2vecModel):
    def construct(self, text):
        _, (hidden, _), _ = self.encoder(text)
        context = ops.concat((hidden[-2, :, :], hidden[-1, :, :]), axis=1)
        output = self.head(context)
        return output

In [5]:
# Hyper-parameters
hidden_size = 256
output_size = 2
num_layers = 2
bidirectional = True
dropout = 0.5
lr = 5e-4

In [6]:
# build encoder
embedding = Glove.from_pretrained('6B', 100, special_tokens=["<unk>", "<pad>"])
lstm_layer = StaticLSTM(100, hidden_size, num_layers=num_layers, batch_first=True,
                     dropout=dropout, bidirectional=bidirectional)
encoder = RNNEncoder(embedding, lstm_layer)

# build head
head = nn.SequentialCell([
    nn.Dropout(p=dropout),
    nn.Dense(hidden_size * 2, output_size)
])

# build network
network = SentimentClassification(encoder, head)
loss = nn.CrossEntropyLoss()
optimizer = nn.Adam(network.trainable_params(), learning_rate=lr)

In [7]:
def initialize_weights(m):
    if isinstance(m, nn.Dense):
        m.weight.set_data(initializer('xavier_normal', m.weight.shape, m.weight.dtype))
        m.bias.set_data(initializer('zeros', m.bias.shape, m.bias.dtype))
    elif isinstance(m, StaticLSTM):
        for name, param in m.parameters_and_names():
            if 'bias' in name:
                param.set_data(initializer('zeros', param.shape, param.dtype))
            elif 'weight' in name:
                param.set_data(initializer('orthogonal', param.shape, param.dtype))

In [8]:
network.apply(initialize_weights)

In [9]:
# define metrics
metric = Accuracy()

# define trainer
trainer = Trainer(network=network, train_dataset=imdb_train, eval_dataset=imdb_test, metrics=metric,
                  epochs=5, loss_fn=loss, optimizer=optimizer)
trainer.run(tgt_columns="label")